# This notebook contains the experiments on Banknote dataset with LionForests

In [1]:
from lionforests import LionForests
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np

Firstly, we load the dataset and we set the feature and class names

In [2]:
banknote_datadset = pd.read_csv('https://raw.githubusercontent.com/Kuntal-G/Machine-Learning/master/R-machine-learning/data/banknote-authentication.csv')
feature_names = ['variance','skew','curtosis','entropy']
class_names=['fake banknote','real banknote'] #0: no, 1: yes #or ['not authenticated banknote','authenticated banknote']

We can plot some instances to see the features and their values

In [3]:
banknote_datadset.head()

,variance,skew,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


Moreover, we can use pandas.describe() to see the ranges of each feature. For example, we observe that curtosis's range is -5.286 to 17.927

In [4]:
banknote_datadset.describe()

,variance,skew,curtosis,entropy,class
count,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000
mean,0.433735,1.922353,1.397627,-1.191657,0.444606
std,2.842763,5.869047,4.310030,2.101013,0.497103
min,-7.042100,-13.773100,-5.286100,-8.548200,0.000000
25%,-1.773000,-1.708200,-1.574975,-2.413450,0.000000
50%,0.496180,2.319650,0.616630,-0.586650,0.000000
75%,2.821475,6.814625,3.179250,0.394810,1.000000
max,6.824800,12.951600,17.927400,2.449500,1.000000


Then We extract the train data from the dataframe

In [5]:
X = banknote_datadset.iloc[:, 0:4].values 
y = banknote_datadset.iloc[:, 4].values 

In [6]:
len(X)

1372

We have 1372 instances. We are going to use the build-in GridSearch of LionForests to find and train the best classifier for this dataset

In [7]:
parameters = [{
    'max_depth': [10],
    'max_features': [0.75],
    'bootstrap': [True],
    'min_samples_leaf' : [1],
    'n_estimators': [500]
}]
lf = LionForests(class_names=class_names)
scaler = MinMaxScaler(feature_range=(-1,1))
lf.train(X, y, scaler, feature_names, parameters)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    3.4s remaining:    2.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.5s finished


Now, we can see the best model's parameters

In [8]:
print("Accuracy:",lf.accuracy,", Number of estimators:",lf.number_of_estimators)
print(lf.model)

Accuracy: 0.9942956150872984 , Number of estimators: 500
RandomForestClassifier(max_depth=10, max_features=0.75, n_estimators=500,
                       n_jobs=-1, random_state=0)


Now lets predict and explain in the same time the third instance

In [9]:
lf.following_breadcrumbs(X[2], False, True, False, complexity=4)

['if 2.397<=variance<=6.825 & -0.829<=curtosis<=17.926 then fake banknote',
 500,
 4,
 251,
 2]

And we can extract the rule without reduction:

In [10]:
lf.following_breadcrumbs(X[2], False, False, False, complexity=4)

['if 2.397<=variance<=6.825 & -3.131<=skew<=-2.303 & 1.816<=curtosis<=2.127 & -0.644<=entropy<=0.728 then fake banknote',
 500,
 4,
 500,
 4]

In [11]:
X[2] #feature_names = ['variance','skew','curtosis','entropy']

array([ 3.866  , -2.6383 ,  1.9242 ,  0.10645])

But lets try to change entropy value:

In [12]:
T_X = X[2]
T_X[3] = 1 #Entropy

We can see that the prediction and the explanation

In [13]:
lf.following_breadcrumbs(T_X, False, True, False, complexity=4)

['if 2.397<=variance<=6.825 & -0.829<=curtosis<=17.926 then fake banknote',
 500,
 4,
 251,
 2]